# <center>ベイズ推論の例: 簡単なテキスト解析「森鴎外か夏目漱石か」</center>
Author: 藤原 義久 <yoshi.fujiwara@gmail.com>    
Data: ディレクトリ"data"以下  
- [青空文庫](https://www.aozora.gr.jp/)にある森鴎外と夏目漱石の作品抜粋  
    > data/docs/01.txt | 森鴎外『雁』  
    > data/docs/02.txt | 森鴎外『かのように』  
    > data/docs/03.txt | 森鴎外『鶏』  
    > data/docs/04.txt | 森鴎外『ヰタ・セクスアリス』  
    > data/docs/05.txt | 夏目漱石『永日小品』  
    > data/docs/06.txt | 夏目漱石『硝子戸の中』  
    > data/docs/07.txt | 夏目漱石『思い出す事など』  
    > data/docs/08.txt | 夏目漱石『夢十夜』  

## 形態素解析のツールmecab
- mecab 本家: https://taku910.github.io/mecab/
- mecab 自体のインストール
    - Windows: https://github.com/ikegami-yukino/mecab/releases/tag/v0.996  
    「MeCab 0.996 64bit version (旧)」にあるインストーラ"mecab-0.996-64.exe"を使う  
    注：環境変数 PATH にmecab をインストールしたパスを追加(例: C:\w10\mecab\bin)  
    注：環境変数 MECABRC を新たに追加して設定(例: C:\w10\mecab\etc\mecabrc)
    - Linux: "mecab linux"でググるとUbuntu, CentOS などでのインストール方法が分かる
    - Mac: "mecab mac"でググる(未確認)
- python からmecab を使うパッケージのインストール  
jupyter notebook を新規に開いて以下を実行する(先頭の"!"に注意)
    - Windows:  
    > !pip install mecab-python-windows
    - Linux:  
    > !pip install mecab-python3
    - Mac: Linuxと同じ(?)

### パッケージの読み込み

In [1]:
import MeCab
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import glob

### 準備：形態素解析

In [2]:
# tagger = MeCab.Tagger()  # Windows の場合
tagger = MeCab.Tagger("-r /etc/mecabrc")  # Linux の場合

### 準備：名詞、動詞、形容詞、助詞だけを選んで文書の「ダイジェスト」を作る

In [3]:
# 名詞、動詞、形容詞、助詞だけを選んで文書の「ダイジェスト」を作る関数を定義

def digest_doc(filename):
    with open(filename, encoding="utf-8") as fin:
        s = fin.read()
        node = tagger.parseToNode(s)

    l = []
    while node:
        x = node.feature.split(',')[0]
        if x == "名詞" or x == "動詞" or x == "形容詞" or x == "助詞":
            l.append(node.feature.split(',')[6])  # 原形を用いる
        node = node.next

    return " ".join(l)

### すべての文書について処理

In [4]:
docs = []
for fn in sorted(glob.glob("data/docs/*")):
    print(fn)
    d = digest_doc(fn)
    docs.append(d)

data/docs/01.txt
data/docs/02.txt
data/docs/03.txt
data/docs/04.txt
data/docs/05.txt
data/docs/06.txt
data/docs/07.txt
data/docs/08.txt


#### すべての文書について語とその頻度の表を作る
メモ：テキスト解析で、語とその頻度の表はterm-frequency matrix と呼ばれている  
以下では機械学習の学習用パッケージ scikit-learn からテキスト解析のツールを用いる

In [5]:
count_vec = CountVectorizer()

x = count_vec.fit_transform(np.array(docs))
# 疎な行列として扱われている
# print(type(X))

td = x.toarray()  # term-document matrix

# 出現したすべての語のリスト
terms = count_vec.get_feature_names()
print(terms)

# term-frequency matrix の次元 = 文書数 * 全語数
print(td.shape)

# term-frequency の中身
print(td)

['あい', 'あう', 'あか', 'あかるい', 'あがる', 'あきらめる', 'あきれる', 'あく', 'あける', 'あざやか', 'あすこ', 'あそこ', 'あたり', 'あちこち', 'あっけない', 'あっち', 'あて', 'あてがう', 'あてる', 'あと', 'あながち', 'あなた', 'あびせる', 'あやす', 'あやぶむ', 'あらあらしい', 'あらわれる', 'あり', 'ありがたい', 'ある', 'あるく', 'あれ', 'あんた', 'いい', 'いう', 'いえる', 'いかん', 'いく', 'いくつ', 'いくら', 'いける', 'いさむ', 'いじくる', 'いじる', 'いずれ', 'いたす', 'いただく', 'いただける', 'いたわる', 'いた事', 'いち', 'いっさい', 'いっしょ', 'いっす', 'いつ', 'いつか', 'いぬ', 'いも', 'いや', 'いやいや', 'いやがる', 'いやしい', 'いらいら', 'いらう', 'いらっしゃい', 'いらっしゃる', 'いる', 'いるか', 'いろいろ', 'うかがう', 'うさ', 'うさんい', 'うすい', 'うずくまる', 'うずめる', 'うそ', 'うた', 'うたう', 'うち', 'うっとうしい', 'うねる', 'うの', 'うぶ', 'うまい', 'うむ', 'うる', 'うるさい', 'うろ覚え', 'ええ', 'えな', 'えらい', 'えり', 'える', 'おいで', 'おう', 'おおい', 'おかしい', 'おく', 'おくれる', 'おこす', 'おこる', 'おさまる', 'おしまい', 'おじ', 'おっくう', 'おつもり', 'おと', 'おとなしい', 'おなり', 'おば', 'おばさん', 'おまわりさん', 'おもちゃ', 'おりる', 'おる', 'おれ', 'おろか', 'おろす', 'お上', 'お世話', 'お作り', 'お前', 'お嬢さん', 'お客', 'お方', 'お次', 'お正月', 'お歯黒', 'お母さん', 'お母様', 'お玉', 'お目見え', 'お祭', 'お笑い', 'お蔭', 'お辞儀', 'かい', 'かう', 'かお', 'かかる', 'かかわ

In [6]:
# pandas のデータフレームに変換する

df_td = pd.DataFrame(data=td, columns=terms)
df_td

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
0,0,0,0,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
3,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,6,1,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,6,0,0,0,0,1,0


In [7]:
# 1番目の文書(森鴎外)について、出現頻度によって語をソート
i = 0
df_td[i:i+1].sort_values(by=i, axis=1, ascending=False)

,する,いる,云う,なる,から,ある,よう,見る,こと,それ,...,家中,宵の口,実生活,害する,宮本,室町,室内,宣告,実行,鼻糞
0,217,190,117,92,89,78,61,52,52,44,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 5番目の文書(夏目漱石)について、出現頻度によって語をソート
i = 4
df_td[i:i+1].sort_values(by=i, axis=1, ascending=False)

,いる,する,から,自分,云う,よう,ある,来る,なる,出る,...,一筋,愛す,愛らしい,愛宕,愛情,愛想,愛敬,愛読,一瞥,鼻糞
4,201,163,104,91,82,53,52,48,47,37,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# 各文書について、頻度の合計を計算
df_td.sum(axis=1)

0    3784
1    3923
2    3813
3    3744
4    3717
5    3670
6    3807
7    3429
dtype: int64

### 「学習用」データを作る

In [10]:
df_td_train = df_td.iloc[[0,1,4,5]]  # 0,1(森鴎外); 4,5(夏目漱石)
df_td_train

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
0,0,0,0,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,1,0,0,0
4,2,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,6,1,0,0,0,1,0


#### 各作家について，語ごとの出現頻度を合計する

In [11]:
# 森鴎外
x = df_td_train[0:2].sum()
freq_ogai = pd.DataFrame(x,columns=["ogai"]).transpose()
freq_ogai

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
ogai,0,0,0,0,1,1,1,0,1,0,...,0,0,1,2,0,0,1,0,1,0


In [12]:
# 夏目漱石
x = df_td_train[2:4].sum()
freq_soseki = pd.DataFrame(x,columns=["soseki"]).transpose()
freq_soseki

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
soseki,2,0,1,1,1,0,0,0,0,1,...,0,0,0,7,1,0,0,1,1,0


In [13]:
# データフレームを連結する
freqs_df = pd.concat([freq_ogai, freq_soseki])
freqs_df

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
ogai,0,0,0,0,1,1,1,0,1,0,...,0,0,1,2,0,0,1,0,1,0
soseki,2,0,1,1,1,0,0,0,0,1,...,0,0,0,7,1,0,0,1,1,0


In [14]:
# 各文書について、語ごとの出現確率を計算
# (1) 頻度が0の場合は確率も0とするナイーブな方法

freqs = np.array(freqs_df, np.float)
freq_sums = np.array(freqs_df.sum(axis=1), np.float).reshape(2,1)  # For numpy's broadcast

probs = freqs / freq_sums

for i in range(probs.shape[1]):
    print("%s\t%f\t%f" % (terms[i],probs[0,i],probs[1,i]))

あい	0.000000	0.000271
あう	0.000000	0.000000
あか	0.000000	0.000135
あかるい	0.000000	0.000135
あがる	0.000130	0.000135
あきらめる	0.000130	0.000000
あきれる	0.000130	0.000000
あく	0.000000	0.000000
あける	0.000130	0.000000
あざやか	0.000000	0.000135
あすこ	0.000000	0.000000
あそこ	0.000000	0.000000
あたり	0.000779	0.000135
あちこち	0.000000	0.000000
あっけない	0.000000	0.000000
あっち	0.000260	0.000000
あて	0.000130	0.000000
あてがう	0.000000	0.000135
あてる	0.000000	0.000000
あと	0.000000	0.001083
あながち	0.000000	0.000000
あなた	0.000000	0.002978
あびせる	0.000130	0.000000
あやす	0.000000	0.000000
あやぶむ	0.000130	0.000000
あらあらしい	0.000130	0.000000
あらわれる	0.000000	0.000135
あり	0.000130	0.000000
ありがたい	0.000000	0.000271
ある	0.018036	0.012996
あるく	0.000000	0.000000
あれ	0.000519	0.000135
あんた	0.000000	0.000000
いい	0.000000	0.000677
いう	0.000000	0.002031
いえる	0.000000	0.000135
いかん	0.000000	0.000000
いく	0.000260	0.000000
いくつ	0.000000	0.000135
いくら	0.000000	0.000000
いける	0.000260	0.001083
いさむ	0.002076	0.000000
いじくる	0.000000	0.000135
いじる	0.000130	0.000000
いずれ	0.000000	0.000135
いた

前方	0.000130	0.000000
前日	0.000000	0.000000
前足	0.000000	0.000135
前途	0.000130	0.000000
剥がす	0.000130	0.000000
剥き出し	0.000000	0.000135
剥げる	0.000000	0.000135
割りふる	0.000000	0.000135
割り戻す	0.000000	0.000000
割り込む	0.000000	0.000000
割る	0.000000	0.000000
割合	0.000130	0.000135
創造	0.000000	0.000000
劈頭	0.000130	0.000000
力学	0.000000	0.000000
加える	0.000000	0.000000
加わる	0.000000	0.000000
加州	0.000130	0.000000
加減	0.000130	0.000000
劣る	0.000000	0.000135
助かる	0.000000	0.000000
助教授	0.000130	0.000000
助言	0.000000	0.000271
努める	0.000260	0.000000
努力	0.000649	0.000000
労働	0.000000	0.000000
効果	0.000000	0.000135
勅語	0.000000	0.000000
勇ましい	0.000000	0.000135
勇む	0.000000	0.000135
勇将	0.000000	0.000135
勇気	0.000000	0.000406
勉強	0.000389	0.000135
動かす	0.000260	0.000541
動き	0.000000	0.000000
動き出す	0.000000	0.000000
動く	0.000000	0.002031
動ける	0.000000	0.000135
動作	0.000130	0.000135
動物	0.000130	0.000000
動詞	0.000000	0.000000
勘定	0.000130	0.000135
勝つ	0.000000	0.000135
勝手	0.000260	0.000406
勝手口	0.000000	0.000000
勝負	0.000000	0.000135
募る	0.000000	

放す	0.000000	0.000000
放つ	0.000000	0.000271
放り出す	0.000000	0.000271
放り込む	0.000000	0.000000
放る	0.000000	0.000135
放射	0.000000	0.000000
政府	0.000130	0.000000
政治	0.000649	0.000271
故人	0.000000	0.000000
故郷	0.000260	0.000135
敏捷	0.000000	0.000000
救う	0.000130	0.000000
敗北	0.000000	0.000135
教える	0.000649	0.000541
教わる	0.000000	0.000000
教員	0.000000	0.000000
教場	0.000000	0.000000
教師	0.000000	0.000271
教授	0.000000	0.000000
教義	0.000908	0.000000
教育	0.001168	0.000000
教訓	0.000000	0.000000
散らす	0.000000	0.000135
散らばる	0.000130	0.000000
散る	0.000000	0.000135
散歩	0.001038	0.000135
敬愛	0.000000	0.000135
数え切れる	0.000000	0.000000
数え立てる	0.000000	0.000135
数多	0.000000	0.000135
数寄屋	0.000130	0.000000
敲き	0.000130	0.000000
整う	0.000000	0.000000
整える	0.000000	0.000000
整理	0.000000	0.000135
敷く	0.000130	0.000677
敷居	0.000000	0.000271
敷布団	0.000000	0.000000
敷板	0.000000	0.000000
敷物	0.000000	0.000135
敷石	0.000000	0.000000
敷金	0.000130	0.000000
文化	0.000130	0.000000
文句	0.000000	0.000406
文士	0.000130	0.000000
文字	0.000000	0.000541
文学	0.000260	0.00

落ちつく	0.000000	0.000271
落ちつける	0.000000	0.000135
落ちる	0.000130	0.000812
落ち合う	0.000130	0.000135
落ち着く	0.000000	0.000135
落ち込む	0.000000	0.000135
葉巻	0.000649	0.000000
葉書	0.000130	0.000000
著い	0.001168	0.000135
著作	0.000000	0.000135
著書	0.000000	0.000000
著者	0.000000	0.000000
著述	0.000000	0.000000
葡萄	0.000130	0.000000
葬式	0.000000	0.000000
葬祭	0.000130	0.000000
葭簀	0.000000	0.000000
蒔く	0.000130	0.000000
蒟蒻	0.000000	0.000000
蒲団	0.000000	0.000406
蒸し暑い	0.000000	0.000000
蒸す	0.000130	0.000000
蒸暑い	0.000000	0.000000
蒼い	0.000389	0.000135
蕎麦	0.000260	0.000000
蕎麦湯	0.000000	0.000406
蕪村	0.000000	0.000000
薄い	0.000260	0.000271
薄れる	0.000000	0.000135
薄井	0.000000	0.000000
薄墨	0.000000	0.000000
薄暗い	0.000260	0.000406
薄曇	0.000260	0.000000
薬礼	0.000000	0.000135
薬缶	0.000000	0.000000
藁沓	0.000000	0.000000
藍染	0.000130	0.000000
藤堂	0.000130	0.000000
藤蔓	0.000000	0.000000
藩屏	0.000130	0.000000
蘇枋	0.000000	0.000000
虎吉	0.000000	0.000000
虚偽	0.000130	0.000000
虚子	0.000000	0.001760
蚊屋	0.000130	0.000000
蚊帳	0.000000	0.000000
蚊遣	0.000130	0.00

In [15]:
# 各文書について、語ごとの出現確率を計算
# (2) 頻度が0の場合は1として扱って確率は0にならないようにする方法

freqs = np.array(freqs_df, np.float)
freq_sums = np.array(freqs_df.sum(axis=1), np.float).reshape(2,1)  # For numpy's broadcast

probs = (freqs + 1.0) / (freq_sums + len(terms))

for i in range(probs.shape[1]):
    print("%s\t%f\t%f" % (terms[i],probs[0,i],probs[1,i]))

あい	0.000076	0.000234
あう	0.000076	0.000078
あか	0.000076	0.000156
あかるい	0.000076	0.000156
あがる	0.000152	0.000156
あきらめる	0.000152	0.000078
あきれる	0.000152	0.000078
あく	0.000076	0.000078
あける	0.000152	0.000078
あざやか	0.000076	0.000156
あすこ	0.000076	0.000078
あそこ	0.000076	0.000078
あたり	0.000533	0.000156
あちこち	0.000076	0.000078
あっけない	0.000076	0.000078
あっち	0.000228	0.000078
あて	0.000152	0.000078
あてがう	0.000076	0.000156
あてる	0.000076	0.000078
あと	0.000076	0.000702
あながち	0.000076	0.000078
あなた	0.000076	0.001794
あびせる	0.000152	0.000078
あやす	0.000076	0.000078
あやぶむ	0.000152	0.000078
あらあらしい	0.000152	0.000078
あらわれる	0.000076	0.000156
あり	0.000152	0.000078
ありがたい	0.000076	0.000234
ある	0.010652	0.007565
あるく	0.000076	0.000078
あれ	0.000380	0.000156
あんた	0.000076	0.000078
いい	0.000076	0.000468
いう	0.000076	0.001248
いえる	0.000076	0.000156
いかん	0.000076	0.000078
いく	0.000228	0.000078
いくつ	0.000076	0.000156
いくら	0.000076	0.000078
いける	0.000228	0.000702
いさむ	0.001293	0.000078
いじくる	0.000076	0.000156
いじる	0.000152	0.000078
いずれ	0.000076	0.000156
いた

マッチ	0.000152	0.000078
マルセイユ	0.000152	0.000078
マントルピース	0.000076	0.000156
メルトン	0.000076	0.000156
ランプ	0.000076	0.000078
リス	0.000076	0.000234
レモン	0.000152	0.000078
レンズ	0.000076	0.000156
ロシア	0.000228	0.000078
ロッキング	0.000076	0.000156
一つ	0.001065	0.000624
一ぱい	0.000152	0.000078
一人前	0.000076	0.000078
一人口	0.000152	0.000078
一人娘	0.000304	0.000078
一代	0.000076	0.000156
一体	0.000380	0.000078
一切	0.000304	0.000078
一刻	0.000076	0.000156
一口	0.000152	0.000078
一口噺	0.000076	0.000078
一句	0.000152	0.000156
一同	0.000076	0.000078
一喜一憂	0.000076	0.000078
一基	0.000076	0.000078
一声	0.000076	0.000078
一夜	0.000076	0.000078
一室	0.000076	0.000078
一家	0.000152	0.000156
一寸	0.000152	0.000156
一座	0.000152	0.000078
一心	0.000076	0.000078
一心不乱	0.000076	0.000078
一昨日	0.000076	0.000234
一時	0.000380	0.000156
一月	0.000076	0.000078
一枝	0.000152	0.000078
一様	0.000076	0.000156
一段	0.000076	0.000078
一段落	0.000076	0.000156
一派	0.000076	0.000078
一片	0.000076	0.000234
一生	0.000076	0.000078
一生懸命	0.000076	0.000078
一番	0.000533	0.000468
一疋	0.000076	0.000078
一目	

墜ちる	0.000076	0.000078
壁画	0.000076	0.000078
壁訴訟	0.000152	0.000078
壁際	0.000152	0.000078
壊乱	0.000152	0.000078
士官	0.000076	0.000078
声色	0.000152	0.000078
声量	0.000076	0.000078
壱岐	0.000076	0.000078
売る	0.000457	0.000078
売れる	0.000076	0.000078
変える	0.000076	0.000078
変じる	0.000076	0.000078
変る	0.000457	0.000468
変動	0.000152	0.000078
変化	0.000076	0.000234
変形	0.000076	0.000078
変態	0.000076	0.000078
夏座敷	0.000152	0.000078
夏目	0.000076	0.000078
夏蜜柑	0.000076	0.000078
夕凪	0.000076	0.000078
夕方	0.000304	0.000078
夕日	0.000228	0.000078
夕食	0.000228	0.000078
外れる	0.000076	0.000312
外交	0.000152	0.000078
外国	0.000152	0.000156
外壁	0.000076	0.000078
外套	0.000076	0.000234
外道	0.000076	0.000078
外部	0.000076	0.000078
多い	0.000380	0.000234
多く	0.000152	0.000546
多事	0.000076	0.000156
多元的	0.000076	0.000078
多分	0.000076	0.000078
多忙	0.000076	0.000156
多数	0.000457	0.000078
多様	0.000076	0.000078
多義	0.000076	0.000078
多量	0.000076	0.000078
夜の目	0.000076	0.000078
夜中	0.000076	0.000234
夜具	0.000076	0.000078
夜明	0.000076	0.000078
夜昼	0.000076	0.000078
夜目	

帰り	0.000304	0.000312
帰る	0.002206	0.002184
帰れる	0.000076	0.000078
帰京	0.000076	0.000078
帰省	0.000152	0.000078
帰着	0.000076	0.000078
帳場	0.000228	0.000078
帷子	0.000076	0.000078
常人	0.000076	0.000078
常住	0.000076	0.000078
常磐	0.000076	0.000078
帽子	0.000076	0.000156
幅広い	0.000152	0.000078
干す	0.000152	0.000078
干る	0.000076	0.000234
平ら	0.000076	0.000078
平仄	0.000076	0.000078
平凡	0.000076	0.000234
平和	0.000228	0.000156
平地	0.000152	0.000078
平家	0.000152	0.000078
平明	0.000152	0.000078
平服	0.000152	0.000078
平気	0.000228	0.000312
平生	0.000380	0.000234
平面	0.000076	0.000078
年中	0.000076	0.000156
年増	0.000152	0.000078
年始	0.000076	0.000156
年寄	0.000076	0.000156
年数	0.000076	0.000078
年来	0.000076	0.000078
年輩	0.000076	0.000078
年長	0.000076	0.000078
年間	0.000076	0.000156
幸い	0.000076	0.000312
幸福	0.000152	0.000078
幽霊	0.000076	0.000156
幾つ	0.000152	0.000078
幾ら	0.000152	0.000078
幾分	0.000228	0.000234
幾年	0.000076	0.000234
幾度	0.000076	0.000078
広い	0.000380	0.000312
広がる	0.000076	0.000156
広げる	0.000228	0.000078
広小路	0.000380	0.000078
広縁	0.000

民族	0.000152	0.000078
気がかり	0.000076	0.000078
気がつく	0.000076	0.000468
気に入る	0.000152	0.000156
気の毒	0.000380	0.000312
気分	0.000076	0.000468
気力	0.000076	0.000078
気味	0.000076	0.000312
気性	0.000076	0.000078
気持	0.000152	0.000078
気楽	0.000076	0.000078
気脈	0.000076	0.000078
気色	0.000076	0.000234
気遣う	0.000152	0.000078
気風	0.000076	0.000078
気高い	0.000076	0.000156
水上	0.000076	0.000234
水仙	0.000076	0.000312
水夫	0.000076	0.000078
水泳	0.000076	0.000156
水浅葱	0.000076	0.000078
水素	0.000076	0.000078
水菓子	0.000076	0.000078
水蜜	0.000076	0.000078
水道	0.000076	0.000156
水際	0.000076	0.000156
氷る	0.000152	0.000078
永い	0.000076	0.000078
永久	0.000076	0.000156
永劫	0.000076	0.000078
永眠	0.000076	0.000078
永続	0.000076	0.000156
求める	0.000152	0.000078
汚い	0.000076	0.000078
汚す	0.000076	0.000312
汚れる	0.000152	0.000234
江戸	0.000152	0.000078
池辺	0.000076	0.000078
汲む	0.000076	0.000078
決心	0.000228	0.000234
汽車	0.000076	0.000078
沈む	0.000152	0.000078
沈没	0.000152	0.000078
沈滞	0.000152	0.000078
沈黙	0.000152	0.000078
沙汰	0.000076	0.000078
没交渉	0.000228	0.000078

直る	0.000457	0.000078
直前	0.000076	0.000078
直接	0.000076	0.000156
直立	0.000076	0.000156
直覚	0.000228	0.000078
直観	0.000076	0.000078
相乗	0.000152	0.000078
相互	0.000076	0.000078
相容れない	0.000152	0.000078
相当	0.000076	0.000234
相応	0.000228	0.000078
相応しい	0.000076	0.000156
相手	0.000457	0.000780
相撲	0.000076	0.000234
相続	0.000152	0.000078
相談	0.000304	0.000234
相貌	0.000076	0.000078
相違	0.000076	0.000234
省みる	0.000152	0.000078
眉山	0.000152	0.000078
眉毛	0.000076	0.000078
眉間	0.000152	0.000078
看護	0.000076	0.000312
県庁	0.000076	0.000078
真っ白い	0.000152	0.000078
真っ赤	0.000152	0.000078
真ん中	0.000152	0.000078
真ん前	0.000228	0.000078
真上	0.000076	0.000078
真中	0.000076	0.000468
真似	0.000380	0.000156
真偽	0.000152	0.000078
真先	0.000076	0.000156
真剣	0.000076	0.000156
真向	0.000076	0.000078
真向い	0.000152	0.000078
真夜中	0.000076	0.000156
真実	0.000152	0.000078
真暗	0.000076	0.000234
真珠貝	0.000076	0.000078
真白	0.000076	0.000078
真直	0.000076	0.000078
真砂	0.000076	0.000156
真綿	0.000152	0.000078
真裸	0.000076	0.000078
真赤	0.000076	0.000078
真逆様	0.000076	0.00007

通勤	0.000152	0.000156
通常	0.000076	0.000078
逝く	0.000076	0.000078
逞しい	0.000076	0.000078
速断	0.000076	0.000078
造る	0.000152	0.000078
造兵	0.000076	0.000156
造次	0.000076	0.000078
逢う	0.000228	0.000156
逢引	0.000076	0.000078
連なる	0.000076	0.000156
連れ	0.000152	0.000078
連れる	0.000533	0.000312
連れ添う	0.000076	0.000156
連れ立つ	0.000076	0.000156
連中	0.000076	0.000468
連帯	0.000076	0.000078
連想	0.000076	0.000078
連署	0.000152	0.000078
週間	0.000152	0.000624
進む	0.000304	0.000078
進める	0.000152	0.000078
進化	0.000076	0.000078
進歩	0.000228	0.000078
進行	0.000076	0.000078
逸する	0.000076	0.000078
遂げる	0.000152	0.000078
遅い	0.000152	0.000234
遇う	0.000076	0.000156
遇する	0.000152	0.000078
遊び	0.000076	0.000390
遊ぶ	0.000304	0.000546
遊山	0.000152	0.000078
遊廓	0.000076	0.000078
運ぶ	0.000228	0.000078
運動	0.000304	0.000078
運命	0.000076	0.000078
運慶	0.000076	0.000078
運行	0.000076	0.000078
過ぎ	0.000152	0.000156
過ぎる	0.000837	0.000546
過す	0.000076	0.000156
過る	0.000076	0.000312
過去	0.000152	0.000468
過渡	0.000152	0.000078
過程	0.000076	0.000078
道ばた	0.000076	0.000078


In [16]:
# 各文書について，出現確率の合計は1になるはず
probs.sum(axis=1)

array([1., 1.])

### 「テスト用」データを作る

In [17]:
df_td_test = df_td.iloc[[2,3,6,7]]
df_td_test

,あい,あう,あか,あかるい,あがる,あきらめる,あきれる,あく,あける,あざやか,...,黒船,黒雲,黙り込む,黙る,黙然,黙許,鼈甲,鼓膜,鼠色,鼻糞
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
3,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,6,0,0,0,0,1,0


In [18]:
freqs = np.array(df_td_test, np.float)
freqs.shape

(4, 5436)

### 尤度(likelihood)の対数をそれぞれのモデル(作家)の場合に計算する

In [19]:
ll = np.dot(freqs, np.log(probs.T))
ll

array([[-27768.81926869, -28175.17554912],
       [-26742.9561593 , -27142.98133145],
       [-29156.93288585, -28371.13203413],
       [-25254.46151005, -24544.39640224]])

#### テスト用の各文書について，どちらのモデル(作家)があてはまるか

In [20]:
writers = ["森鴎外","夏目漱石"]
for k in np.argmax(ll,axis=1):
    print(writers[k])

森鴎外
森鴎外
夏目漱石
夏目漱石
